# Extension Methods - Good vs Bad Examples

This notebook demonstrates **meaningful** vs **trivial** implementations of extension methods in C#.

## What Are Extension Methods?

Extension methods allow you to "add" methods to existing types without modifying their source code or creating derived types. They're particularly useful for:

- Adding functionality to **built-in types** (string, int, DateTime, etc.)
- Extending **third-party library types** you can't modify
- Adding **LINQ-style** fluent methods to collections
- Creating **domain-specific** helper methods

### ✅ Good Examples - Meaningful Extension Method Usage

In [ ]:
// ✅ GOOD: Extensions for built-in types that add domain-specific functionality
public static class StringExtensions
{
    // Email validation extension
    public static bool IsValidEmail(this string email)
    {
        if (string.IsNullOrWhiteSpace(email))
            return false;
            
        try
        {
            var addr = new System.Net.Mail.MailAddress(email);
            return addr.Address == email;
        }
        catch
        {
            return false;
        }
    }
    
    // Truncate with ellipsis
    public static string Truncate(this string value, int maxLength, string suffix = "...")
    {
        if (string.IsNullOrEmpty(value))
            return value;
            
        if (value.Length <= maxLength)
            return value;
            
        return value.Substring(0, maxLength - suffix.Length) + suffix;
    }
    
    // Convert to title case with business rules
    public static string ToTitleCase(this string input)
    {
        if (string.IsNullOrEmpty(input))
            return input;
            
        var textInfo = System.Globalization.CultureInfo.CurrentCulture.TextInfo;
        var result = textInfo.ToTitleCase(input.ToLower());
        
        // Business rule: Keep certain words lowercase
        var lowercaseWords = new[] { "of", "the", "and", "or", "but", "in", "on", "at", "to", "for" };
        foreach (var word in lowercaseWords)
        {
            result = System.Text.RegularExpressions.Regex.Replace(result, 
                $@"\b{word}\b", word, System.Text.RegularExpressions.RegexOptions.IgnoreCase);
        }
        
        return result;
    }
    
    // Safe substring that doesn't throw exceptions
    public static string SafeSubstring(this string input, int startIndex, int length)
    {
        if (string.IsNullOrEmpty(input) || startIndex >= input.Length || startIndex < 0)
            return string.Empty;
            
        if (startIndex + length > input.Length)
            length = input.Length - startIndex;
            
        return input.Substring(startIndex, length);
    }
}

In [ ]:
// ✅ GOOD USAGE: String extensions add meaningful functionality

// Example 1: Email validation
var emails = new[] { "test@example.com", "invalid-email", "user@domain.co.uk", "" };
foreach (var email in emails)
{
    Console.WriteLine($"{email,-20} -> Valid: {email.IsValidEmail()}");
}
Console.WriteLine();

// Example 2: Text truncation for UI display
var longText = "This is a very long text that needs to be truncated for display purposes";
Console.WriteLine($"Original: {longText}");
Console.WriteLine($"Truncated: {longText.Truncate(30)}");
Console.WriteLine($"Custom suffix: {longText.Truncate(25, " (more...)")}");
Console.WriteLine();

// Example 3: Title case with business rules
var titles = new[] { "lord of the rings", "WAR AND PEACE", "the catcher in the rye" };
foreach (var title in titles)
{
    Console.WriteLine($"'{title}' -> '{title.ToTitleCase()}'" );
}
Console.WriteLine();

// Example 4: Safe substring operations
var testString = "Hello World";
Console.WriteLine($"Safe substring (0, 5): '{testString.SafeSubstring(0, 5)}'");
Console.WriteLine($"Safe substring (6, 10): '{testString.SafeSubstring(6, 10)}'");
Console.WriteLine($"Safe substring (20, 5): '{testString.SafeSubstring(20, 5)}'"); // Won't throw

In [ ]:
// ✅ GOOD: Extensions for DateTime with business logic
public static class DateTimeExtensions
{
    // Business day calculations
    public static bool IsBusinessDay(this DateTime date)
    {
        return date.DayOfWeek != DayOfWeek.Saturday && date.DayOfWeek != DayOfWeek.Sunday;
    }
    
    public static DateTime NextBusinessDay(this DateTime date)
    {
        do
        {
            date = date.AddDays(1);
        }
        while (!date.IsBusinessDay());
        
        return date;
    }
    
    public static int BusinessDaysUntil(this DateTime startDate, DateTime endDate)
    {
        if (startDate >= endDate)
            return 0;
            
        int businessDays = 0;
        var current = startDate;
        
        while (current < endDate)
        {
            if (current.IsBusinessDay())
                businessDays++;
            current = current.AddDays(1);
        }
        
        return businessDays;
    }
    
    // Age calculation
    public static int CalculateAge(this DateTime birthDate, DateTime? asOfDate = null)
    {
        var referenceDate = asOfDate ?? DateTime.Today;
        
        var age = referenceDate.Year - birthDate.Year;
        
        // Subtract a year if birthday hasn't occurred this year
        if (referenceDate.Month < birthDate.Month || 
            (referenceDate.Month == birthDate.Month && referenceDate.Day < birthDate.Day))
        {
            age--;
        }
        
        return age;
    }
    
    // Friendly time descriptions
    public static string ToFriendlyString(this DateTime dateTime)
    {
        var timeSpan = DateTime.Now - dateTime;
        
        if (timeSpan.TotalMinutes < 1)
            return "just now";
        if (timeSpan.TotalMinutes < 60)
            return $"{(int)timeSpan.TotalMinutes} minute(s) ago";
        if (timeSpan.TotalHours < 24)
            return $"{(int)timeSpan.TotalHours} hour(s) ago";
        if (timeSpan.TotalDays < 7)
            return $"{(int)timeSpan.TotalDays} day(s) ago";
        if (timeSpan.TotalDays < 30)
            return $"{(int)(timeSpan.TotalDays / 7)} week(s) ago";
        if (timeSpan.TotalDays < 365)
            return $"{(int)(timeSpan.TotalDays / 30)} month(s) ago";
        
        return $"{(int)(timeSpan.TotalDays / 365)} year(s) ago";
    }
}

In [ ]:
// Usage examples for DateTime extensions
var today = DateTime.Today;
var birthday = new DateTime(1990, 6, 15);
var projectDeadline = today.AddDays(10);

// Example 1: Business day calculations
Console.WriteLine($"Today is business day: {today.IsBusinessDay()}");
Console.WriteLine($"Next business day: {today.NextBusinessDay():yyyy-MM-dd}");
Console.WriteLine($"Business days until deadline: {today.BusinessDaysUntil(projectDeadline)}");
Console.WriteLine();

// Example 2: Age calculation
Console.WriteLine($"Age today: {birthday.CalculateAge()}");
Console.WriteLine($"Age on 2020-01-01: {birthday.CalculateAge(new DateTime(2020, 1, 1))}");
Console.WriteLine();

// Example 3: Friendly time descriptions
var timePoints = new[]
{
    DateTime.Now.AddMinutes(-30),
    DateTime.Now.AddHours(-2),
    DateTime.Now.AddDays(-3),
    DateTime.Now.AddDays(-15),
    DateTime.Now.AddDays(-100)
};

foreach (var time in timePoints)
{
    Console.WriteLine($"{time:yyyy-MM-dd HH:mm} -> {time.ToFriendlyString()}");
}

In [ ]:
// ✅ GOOD: LINQ-style extensions for collections
public static class CollectionExtensions
{
    // Batch processing for large collections
    public static IEnumerable<IEnumerable<T>> Batch<T>(this IEnumerable<T> source, int batchSize)
    {
        if (batchSize <= 0)
            throw new ArgumentException("Batch size must be positive", nameof(batchSize));
            
        var batch = new List<T>(batchSize);
        
        foreach (var item in source)
        {
            batch.Add(item);
            
            if (batch.Count == batchSize)
            {
                yield return batch;
                batch = new List<T>(batchSize);
            }
        }
        
        if (batch.Count > 0)
            yield return batch;
    }
    
    // Safe operations that handle null collections
    public static bool IsNullOrEmpty<T>(this IEnumerable<T> source)
    {
        return source == null || !source.Any();
    }
    
    public static IEnumerable<T> OrEmpty<T>(this IEnumerable<T> source)
    {
        return source ?? Enumerable.Empty<T>();
    }
    
    // Distinct by property
    public static IEnumerable<T> DistinctBy<T, TKey>(this IEnumerable<T> source, Func<T, TKey> keySelector)
    {
        var seenKeys = new HashSet<TKey>();
        foreach (var element in source)
        {
            if (seenKeys.Add(keySelector(element)))
            {
                yield return element;
            }
        }
    }
    
    // Random selection
    public static T RandomElement<T>(this IEnumerable<T> source)
    {
        var list = source.ToList();
        if (list.Count == 0)
            throw new InvalidOperationException("Sequence contains no elements");
            
        var random = new Random();
        return list[random.Next(list.Count)];
    }
    
    public static IEnumerable<T> Shuffle<T>(this IEnumerable<T> source)
    {
        var list = source.ToList();
        var random = new Random();
        
        for (int i = list.Count - 1; i > 0; i--)
        {
            int j = random.Next(i + 1);
            yield return list[j];
            list[j] = list[i];
        }
        
        if (list.Count > 0)
            yield return list[0];
    }
}

In [ ]:
// Usage examples for collection extensions
var numbers = Enumerable.Range(1, 20).ToList();
var people = new[]
{
    new { Name = "John", Department = "IT" },
    new { Name = "Jane", Department = "HR" },
    new { Name = "Bob", Department = "IT" },
    new { Name = "Alice", Department = "Finance" }
};

// Example 1: Batch processing
Console.WriteLine("Batch processing:");
foreach (var batch in numbers.Batch(5))
{
    Console.WriteLine($"Batch: [{string.Join(", ", batch)}]");
}
Console.WriteLine();

// Example 2: Null-safe operations
List<int> nullList = null;
var emptyList = new List<int>();

Console.WriteLine($"Null list is null or empty: {nullList.IsNullOrEmpty()}");
Console.WriteLine($"Empty list is null or empty: {emptyList.IsNullOrEmpty()}");
Console.WriteLine($"Numbers list is null or empty: {numbers.IsNullOrEmpty()}");
Console.WriteLine($"Null list or empty count: {nullList.OrEmpty().Count()}");
Console.WriteLine();

// Example 3: Distinct by property
Console.WriteLine("Distinct departments:");
var distinctDepartments = people.DistinctBy(p => p.Department);
foreach (var person in distinctDepartments)
{
    Console.WriteLine($"{person.Name} - {person.Department}");
}
Console.WriteLine();

// Example 4: Random operations
Console.WriteLine($"Random number: {numbers.RandomElement()}");
Console.WriteLine($"Random person: {people.RandomElement().Name}");
Console.WriteLine($"Shuffled first 5: [{string.Join(", ", numbers.Shuffle().Take(5))}]");

In [ ]:
// ✅ GOOD: Extensions for third-party or system types you can't modify
public static class HttpExtensions
{
    // Extension for HttpClient (third-party-like scenario)
    public static async Task<T> GetJsonAsync<T>(this HttpClient client, string requestUri)
    {
        var response = await client.GetAsync(requestUri);
        response.EnsureSuccessStatusCode();
        
        var json = await response.Content.ReadAsStringAsync();
        return System.Text.Json.JsonSerializer.Deserialize<T>(json);
    }
    
    public static async Task<HttpResponseMessage> PostJsonAsync<T>(this HttpClient client, 
                                                                  string requestUri, T data)
    {
        var json = System.Text.Json.JsonSerializer.Serialize(data);
        var content = new StringContent(json, System.Text.Encoding.UTF8, "application/json");
        
        return await client.PostAsync(requestUri, content);
    }
}

// Extensions for Exception handling
public static class ExceptionExtensions
{
    // Get full exception details including inner exceptions
    public static string GetFullMessage(this Exception exception)
    {
        var messages = new List<string>();
        var current = exception;
        
        while (current != null)
        {
            messages.Add(current.Message);
            current = current.InnerException;
        }
        
        return string.Join(" -> ", messages);
    }
    
    // Check if exception or inner exceptions contain specific type
    public static bool ContainsException<T>(this Exception exception) where T : Exception
    {
        var current = exception;
        while (current != null)
        {
            if (current is T)
                return true;
            current = current.InnerException;
        }
        return false;
    }
}

// Extensions for decimal/money calculations
public static class DecimalExtensions
{
    public static decimal CalculateTax(this decimal amount, decimal taxRate)
    {
        return Math.Round(amount * taxRate, 2, MidpointRounding.AwayFromZero);
    }
    
    public static decimal WithTax(this decimal amount, decimal taxRate)
    {
        return amount + amount.CalculateTax(taxRate);
    }
    
    public static string ToCurrency(this decimal amount, string currencySymbol = "$")
    {
        return $"{currencySymbol}{amount:N2}";
    }
}

In [ ]:
// Usage examples for system type extensions
// Note: HttpClient examples would need actual HTTP endpoints to work

// Example 1: Exception handling
try
{
    throw new InvalidOperationException("Outer exception", 
        new ArgumentException("Inner exception", 
            new NullReferenceException("Deep inner exception")));
}
catch (Exception ex)
{
    Console.WriteLine($"Full message: {ex.GetFullMessage()}");
    Console.WriteLine($"Contains ArgumentException: {ex.ContainsException<ArgumentException>()}");
    Console.WriteLine($"Contains FileNotFoundException: {ex.ContainsException<FileNotFoundException>()}");
}
Console.WriteLine();

// Example 2: Decimal/money calculations
var price = 99.99m;
var taxRate = 0.08m; // 8% tax

Console.WriteLine($"Price: {price.ToCurrency()}");
Console.WriteLine($"Tax: {price.CalculateTax(taxRate).ToCurrency()}");
Console.WriteLine($"Total with tax: {price.WithTax(taxRate).ToCurrency()}");
Console.WriteLine($"In Euros: {price.ToCurrency("€")}");
Console.WriteLine();

// Example 3: Fluent calculations
var items = new[] { 29.99m, 45.50m, 12.99m };
var total = items.Sum();
var finalPrice = total.WithTax(0.10m); // 10% tax

Console.WriteLine($"Subtotal: {total.ToCurrency()}");
Console.WriteLine($"Final price: {finalPrice.ToCurrency()}");

### ❌ Bad Examples - Trivial Extension Method Usage

In [ ]:
// ❌ BAD: Extension methods on your own classes (you can just add the method directly)
public class MyClass
{
    public string Name { get; set; }
    public int Value { get; set; }
}

public static class BadExtensions
{
    // ❌ BAD: Extending your own class - should be a regular method
    public static void PrintInfo(this MyClass obj)
    {
        Console.WriteLine($"Name: {obj.Name}, Value: {obj.Value}");
    }
    // Problem: You own MyClass, so just add PrintInfo() as a regular method
    
    // ❌ BAD: Trivial wrappers that don't add value
    public static bool IsEmpty(this string str)
    {
        return string.IsNullOrEmpty(str);
    }
    // Problem: Just a wrapper around existing functionality
    
    // ❌ BAD: Extensions that just call other methods
    public static string MakeUpper(this string str)
    {
        return str.ToUpper();
    }
    // Problem: ToUpper() already exists and is just as clear
    
    // ❌ BAD: Too simple to be useful
    public static int Double(this int number)
    {
        return number * 2;
    }
    // Problem: number * 2 is simpler and clearer than number.Double()
    
    // ❌ BAD: Confusing or misleading names
    public static int Calculate(this int number)
    {
        return number + 10;
    }
    // Problem: "Calculate" doesn't tell you what it calculates
    
    // ❌ BAD: Extension methods that break conventions
    public static void Print(this object obj, string prefix = "")
    {
        Console.WriteLine(prefix + obj.ToString());
    }
    // Problem: Extending System.Object affects EVERYTHING
    // and Print() suggests output but doesn't say where
}

In [ ]:
// ❌ BAD USAGE: These examples don't demonstrate meaningful extension methods
var myObj = new MyClass { Name = "Test", Value = 42 };
var testString = "Hello World";
var testNumber = 5;

// These calls work but don't show the value of extension methods:
myObj.PrintInfo();                    // Should be a regular method on MyClass
Console.WriteLine(testString.IsEmpty());       // Just use string.IsNullOrEmpty()
Console.WriteLine(testString.MakeUpper());     // Just use ToUpper()
Console.WriteLine(testNumber.Double());        // Just use number * 2
Console.WriteLine(testNumber.Calculate());     // Unclear what this does
myObj.Print("Object: ");             // Affects all objects - too broad

// Problems with these examples:
// 1. Don't solve real problems
// 2. Add unnecessary complexity
// 3. Don't leverage the main benefits of extension methods
// 4. Could be regular methods or simple expressions
// 5. Don't add domain-specific value

## Domain-Specific Extensions

### ✅ Good Examples - Business Domain Extensions

In [ ]:
// ✅ GOOD: Domain-specific extensions for e-commerce
public static class ECommerceExtensions
{
    // Product filtering extensions
    public static IEnumerable<Product> InStock(this IEnumerable<Product> products)
    {
        return products.Where(p => p.StockQuantity > 0);
    }
    
    public static IEnumerable<Product> OnSale(this IEnumerable<Product> products)
    {
        return products.Where(p => p.SalePrice.HasValue && p.SalePrice < p.RegularPrice);
    }
    
    public static IEnumerable<Product> UnderPrice(this IEnumerable<Product> products, decimal maxPrice)
    {
        return products.Where(p => (p.SalePrice ?? p.RegularPrice) <= maxPrice);
    }
    
    public static IEnumerable<Product> ByCategory(this IEnumerable<Product> products, string category)
    {
        return products.Where(p => string.Equals(p.Category, category, StringComparison.OrdinalIgnoreCase));
    }
    
    // Order processing extensions
    public static decimal CalculateShipping(this Order order, string shippingMethod = "standard")
    {
        var total = order.Items.Sum(item => item.Price * item.Quantity);
        
        return shippingMethod.ToLower() switch
        {
            "free" => 0m,
            "standard" => total >= 50m ? 0m : 5.99m,
            "express" => total >= 100m ? 9.99m : 14.99m,
            "overnight" => 24.99m,
            _ => 5.99m
        };
    }
    
    public static bool IsEligibleForFreeShipping(this Order order)
    {
        return order.Items.Sum(item => item.Price * item.Quantity) >= 50m;
    }
    
    public static decimal CalculateTotal(this Order order, string shippingMethod = "standard")
    {
        var subtotal = order.Items.Sum(item => item.Price * item.Quantity);
        var shipping = order.CalculateShipping(shippingMethod);
        var tax = subtotal * 0.08m; // 8% tax
        
        return subtotal + shipping + tax;
    }
}

// Supporting classes for the examples
public class Product
{
    public string Name { get; set; }
    public decimal RegularPrice { get; set; }
    public decimal? SalePrice { get; set; }
    public int StockQuantity { get; set; }
    public string Category { get; set; }
}

public class OrderItem
{
    public string ProductName { get; set; }
    public decimal Price { get; set; }
    public int Quantity { get; set; }
}

public class Order
{
    public List<OrderItem> Items { get; set; } = new();
    public DateTime OrderDate { get; set; }
    public string CustomerEmail { get; set; }
}

In [ ]:
// Usage examples for domain-specific extensions
var products = new List<Product>
{
    new() { Name = "Laptop", RegularPrice = 999.99m, SalePrice = 899.99m, StockQuantity = 5, Category = "Electronics" },
    new() { Name = "Book", RegularPrice = 29.99m, StockQuantity = 0, Category = "Books" },
    new() { Name = "Shirt", RegularPrice = 39.99m, SalePrice = 29.99m, StockQuantity = 10, Category = "Clothing" },
    new() { Name = "Phone", RegularPrice = 699.99m, StockQuantity = 3, Category = "Electronics" },
    new() { Name = "Shoes", RegularPrice = 89.99m, StockQuantity = 7, Category = "Clothing" }
};

// Example 1: Fluent product filtering
Console.WriteLine("Electronics on sale, in stock, under $1000:");
var filteredProducts = products
    .ByCategory("Electronics")
    .OnSale()
    .InStock()
    .UnderPrice(1000m);

foreach (var product in filteredProducts)
{
    Console.WriteLine($"- {product.Name}: ${product.SalePrice ?? product.RegularPrice} (Stock: {product.StockQuantity})");
}
Console.WriteLine();

// Example 2: Order calculations
var order = new Order
{
    CustomerEmail = "customer@example.com",
    OrderDate = DateTime.Now,
    Items = new List<OrderItem>
    {
        new() { ProductName = "Laptop", Price = 899.99m, Quantity = 1 },
        new() { ProductName = "Shirt", Price = 29.99m, Quantity = 2 }
    }
};

Console.WriteLine($"Order eligible for free shipping: {order.IsEligibleForFreeShipping()}");
Console.WriteLine($"Standard shipping cost: ${order.CalculateShipping("standard")}");
Console.WriteLine($"Express shipping cost: ${order.CalculateShipping("express")}");
Console.WriteLine($"Total with standard shipping: ${order.CalculateTotal("standard")}");
Console.WriteLine($"Total with express shipping: ${order.CalculateTotal("express")}");

In [ ]:
// ✅ GOOD: Extensions for data validation and formatting
public static class ValidationExtensions
{
    // Credit card validation
    public static bool IsValidCreditCard(this string creditCard)
    {
        if (string.IsNullOrWhiteSpace(creditCard))
            return false;
            
        // Remove spaces and dashes
        var cleanCard = creditCard.Replace(" ", "").Replace("-", "");
        
        // Check if all digits and correct length
        if (!cleanCard.All(char.IsDigit) || cleanCard.Length < 13 || cleanCard.Length > 19)
            return false;
            
        // Luhn algorithm
        int sum = 0;
        bool alternate = false;
        
        for (int i = cleanCard.Length - 1; i >= 0; i--)
        {
            int digit = int.Parse(cleanCard[i].ToString());
            
            if (alternate)
            {
                digit *= 2;
                if (digit > 9)
                    digit = (digit % 10) + 1;
            }
            
            sum += digit;
            alternate = !alternate;
        }
        
        return sum % 10 == 0;
    }
    
    // Phone number validation and formatting
    public static bool IsValidPhoneNumber(this string phoneNumber)
    {
        if (string.IsNullOrWhiteSpace(phoneNumber))
            return false;
            
        var digits = new string(phoneNumber.Where(char.IsDigit).ToArray());
        return digits.Length == 10 || (digits.Length == 11 && digits[0] == '1');
    }
    
    public static string FormatPhoneNumber(this string phoneNumber)
    {
        if (!phoneNumber.IsValidPhoneNumber())
            return phoneNumber;
            
        var digits = new string(phoneNumber.Where(char.IsDigit).ToArray());
        
        if (digits.Length == 11 && digits[0] == '1')
            digits = digits.Substring(1);
            
        return $"({digits.Substring(0, 3)}) {digits.Substring(3, 3)}-{digits.Substring(6, 4)}";
    }
    
    // Password strength validation
    public static bool IsStrongPassword(this string password)
    {
        if (string.IsNullOrWhiteSpace(password) || password.Length < 8)
            return false;
            
        bool hasUpper = password.Any(char.IsUpper);
        bool hasLower = password.Any(char.IsLower);
        bool hasDigit = password.Any(char.IsDigit);
        bool hasSpecial = password.Any(c => !char.IsLetterOrDigit(c));
        
        return hasUpper && hasLower && hasDigit && hasSpecial;
    }
    
    public static string GetPasswordStrength(this string password)
    {
        if (string.IsNullOrWhiteSpace(password))
            return "Very Weak";
            
        int score = 0;
        
        if (password.Length >= 8) score++;
        if (password.Length >= 12) score++;
        if (password.Any(char.IsUpper)) score++;
        if (password.Any(char.IsLower)) score++;
        if (password.Any(char.IsDigit)) score++;
        if (password.Any(c => !char.IsLetterOrDigit(c))) score++;
        
        return score switch
        {
            0 or 1 => "Very Weak",
            2 or 3 => "Weak",
            4 => "Moderate",
            5 => "Strong",
            6 => "Very Strong",
            _ => "Very Strong"
        };
    }
}

In [ ]:
// Usage examples for validation extensions
var testData = new[]
{
    ("Credit Cards", new[] { "4532-1234-5678-9012", "1234-5678-9012-3456", "invalid-card" }),
    ("Phone Numbers", new[] { "(555) 123-4567", "555.123.4567", "5551234567", "123-456", "15551234567" }),
    ("Passwords", new[] { "password", "Password1", "Pass123!", "MySecureP@ssw0rd!", "abc" })
};

// Example 1: Credit card validation
Console.WriteLine("Credit Card Validation:");
foreach (var card in testData[0].Item2)
{
    Console.WriteLine($"{card,-20} -> Valid: {card.IsValidCreditCard()}");
}
Console.WriteLine();

// Example 2: Phone number validation and formatting
Console.WriteLine("Phone Number Validation and Formatting:");
foreach (var phone in testData[1].Item2)
{
    Console.WriteLine($"{phone,-15} -> Valid: {phone.IsValidPhoneNumber(),-5} Formatted: {phone.FormatPhoneNumber()}");
}
Console.WriteLine();

// Example 3: Password strength analysis
Console.WriteLine("Password Strength Analysis:");
foreach (var password in testData[2].Item2)
{
    Console.WriteLine($"{password,-20} -> Strong: {password.IsStrongPassword(),-5} Strength: {password.GetPasswordStrength()}");
}

## Summary

### What Makes Extension Methods "Meaningful":

**Perfect Use Cases:**
- **Built-in types** (string, DateTime, int, decimal) - Add domain-specific functionality
- **Third-party types** (HttpClient, Exception) - Add convenience methods you can't modify
- **Collection operations** - LINQ-style fluent methods for specific domains
- **Cross-cutting concerns** - Validation, formatting, calculations that apply to multiple types
- **Business domain logic** - E-commerce, financial, validation operations

**Key Benefits:**
1. **Fluent APIs** - Chain operations naturally: `products.InStock().OnSale().UnderPrice(100)`
2. **Domain expressiveness** - `date.NextBusinessDay()` is clearer than `BusinessDateHelper.GetNext(date)`
3. **Type safety** - Extension methods provide compile-time checking
4. **Discoverability** - IntelliSense shows available extensions
5. **Reusability** - Same extensions work across multiple projects

### When Extension Methods Add Real Value:

**Domain-Specific Operations:**
- Business calculations: `amount.WithTax()`, `date.NextBusinessDay()`
- Validation: `email.IsValidEmail()`, `password.IsStrongPassword()`
- Formatting: `phone.FormatPhoneNumber()`, `amount.ToCurrency()`

**Collection Processing:**
- Batch operations: `items.Batch(100)`
- Domain filtering: `products.InStock().OnSale()`
- Safe operations: `collection.OrEmpty()`, `list.IsNullOrEmpty()`

**Cross-Platform Utilities:**
- Error handling: `exception.GetFullMessage()`
- HTTP operations: `client.GetJsonAsync<T>()`
- Date calculations: `birthday.CalculateAge()`

### Red Flags (Avoid These):

**Don't Use Extension Methods For:**
- **Your own classes** - Add methods directly to the class instead
- **Simple wrappers** - `str.IsEmpty()` when `string.IsNullOrEmpty()` exists
- **Trivial operations** - `number.Double()` when `number * 2` is clearer
- **Object extensions** - Extending `System.Object` affects everything
- **Confusing names** - Methods that don't clearly indicate their purpose

**Anti-Patterns:**
- Extensions that just call existing methods without adding value
- Too many extensions that clutter IntelliSense
- Extensions with unclear or misleading names
- Breaking established conventions or patterns
- Extensions that should be separate utility classes

### Best Practices:

1. **Clear naming** - Method names should be self-documenting
2. **Single responsibility** - Each extension should do one thing well
3. **Null safety** - Handle null inputs gracefully
4. **Domain focus** - Group related extensions in themed static classes
5. **Documentation** - Complex extensions need clear XML comments
6. **Performance aware** - Avoid extensions that create unnecessary allocations
7. **Consistent with existing patterns** - Follow LINQ-style conventions when appropriate

### Examples Summary:

**✅ Good Examples Shown:**
- String operations: email validation, truncation, title case
- DateTime extensions: business days, age calculation, friendly formatting
- Collection utilities: batching, null safety, distinct by property
- HTTP client helpers: JSON serialization shortcuts
- Exception handling: full message extraction, type checking
- Financial calculations: tax, currency formatting
- E-commerce domain: product filtering, order calculations
- Validation utilities: credit cards, phone numbers, passwords

These examples demonstrate how extension methods can make code more readable, expressive, and maintainable when used appropriately.